In [1]:
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import OneHotEncoder
import keras
from keras.preprocessing.image import ImageDataGenerator  
from keras.models import Sequential, load_model
from keras.optimizers import SGD, Adam
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, GlobalMaxPooling1D
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from keras.layers import Activation, Dropout, Flatten, Dense  
from keras import backend as K 
from keras import metrics
import os
import time
import h5py

D:\anacondainstall\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(1)

In [3]:
train_doc = np.load('flower_train.npy')
test_doc = np.load('flower_test.npy')

In [4]:
sorted_train_doc = np.array(sorted(train_doc,key=lambda x:x[3]))
sorted_test_doc = np.array(sorted(test_doc,key=lambda x:x[3]))

In [5]:
f_train = h5py.File('res_flower_train.h5','r') #img data is sorted by the filenames
train_img = f_train['res_flower_train']
train_id = sorted_train_doc[:,0]
f_test = h5py.File('res_flower_test.h5','r')
test_img = f_test['res_flower_test']
test_id = sorted_test_doc[:,0]

In [6]:
new_train_doc = sorted_train_doc
uni_train_id = np.unique(train_id)
for i in range(len(uni_train_id)):
    d = uni_train_id[i]
    new_train_doc[:,0][(train_id == d)] = str(i)
train_label = new_train_doc[:,0].astype(np.int)

In [7]:
new_test_doc = sorted_test_doc
uni_test_id = np.unique(test_id)
for i in range(len(uni_test_id)):
    d = uni_test_id[i]
    new_test_doc[:,0][(test_id == d)] = str(i)
test_label = new_test_doc[:,0].astype(np.int)

In [8]:
enc = OneHotEncoder()
enc.fit(np.arange(0,20).reshape(-1,1))
onehot_train_label = enc.transform(np.array(train_label).reshape(-1,1)).toarray()

In [94]:
def top_1(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=1)

In [89]:
def top_2(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=2)

In [90]:
def top_3(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

In [91]:
def top_4(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=4)

In [92]:
def top_5(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)

In [95]:
model = Sequential()
model.add(Flatten(input_shape=(1,1,1,2048)))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='softmax')) 

In [96]:
#sgd = SGD(lr=0.001, decay= 1e-6, momentum=0.9, nesterov=True) 
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy',top_1,top_2,top_3,top_4,top_5,])#,metrics.categorical_accuracy,metrics.top_k_categorical_accuracy])
model.fit(train_img, onehot_train_label,epochs=60,batch_size=100,shuffle="batch")
#60
#80

Epoch 1/60
2128/2128 [==============================] - 1s 700us/step - loss: 3.4997 - acc: 0.0710 - top_1: 0.0710 - top_2: 0.1335 - top_3: 0.1941 - top_4: 0.2570 - top_5: 0.3144
Epoch 2/60
2128/2128 [==============================] - 0s 222us/step - loss: 2.9207 - acc: 0.1217 - top_1: 0.1217 - top_2: 0.2119 - top_3: 0.2965 - top_4: 0.3750 - top_5: 0.4366
Epoch 3/60
2128/2128 [==============================] - 0s 231us/step - loss: 2.6804 - acc: 0.1960 - top_1: 0.1960 - top_2: 0.3073 - top_3: 0.3905 - top_4: 0.4629 - top_5: 0.5277
Epoch 4/60
2128/2128 [==============================] - 1s 243us/step - loss: 2.4664 - acc: 0.2622 - top_1: 0.2622 - top_2: 0.3882 - top_3: 0.4807 - top_4: 0.5587 - top_5: 0.6283
Epoch 5/60
2128/2128 [==============================] - 0s 230us/step - loss: 2.2531 - acc: 0.3398 - top_1: 0.3398 - top_2: 0.4817 - top_3: 0.5714 - top_4: 0.6504 - top_5: 0.7035
Epoch 6/60
2128/2128 [==============================] - 0s 232us/step - loss: 2.0701 - acc: 0.3816 - top_

Epoch 46/60
2128/2128 [==============================] - 1s 391us/step - loss: 0.3040 - acc: 0.9126 - top_1: 0.9126 - top_2: 0.9723 - top_3: 0.9887 - top_4: 0.9930 - top_5: 0.9953
Epoch 47/60
2128/2128 [==============================] - 1s 397us/step - loss: 0.2875 - acc: 0.9196 - top_1: 0.9196 - top_2: 0.9746 - top_3: 0.9883 - top_4: 0.9953 - top_5: 0.9962
Epoch 48/60
2128/2128 [==============================] - 1s 385us/step - loss: 0.2966 - acc: 0.9117 - top_1: 0.9117 - top_2: 0.9718 - top_3: 0.9864 - top_4: 0.9930 - top_5: 0.9944
Epoch 49/60
2128/2128 [==============================] - 1s 364us/step - loss: 0.2516 - acc: 0.9319 - top_1: 0.9319 - top_2: 0.9812 - top_3: 0.9915 - top_4: 0.9962 - top_5: 0.9972
Epoch 50/60
2128/2128 [==============================] - 1s 246us/step - loss: 0.2475 - acc: 0.9352 - top_1: 0.9352 - top_2: 0.9821 - top_3: 0.9901 - top_4: 0.9958 - top_5: 0.9972
Epoch 51/60
2128/2128 [==============================] - 0s 232us/step - loss: 0.2392 - acc: 0.9319 

In [114]:
predicted_pro = model.predict(test_img)
train_predicted_pro = model.predict(train_img)

In [115]:
train_predicted_class = []
for i in range(train_predicted_pro.shape[0]):
    best_prediction = np.argsort(train_predicted_pro[i])[::-1][0] #plus one to get the class label but not the index
    train_predicted_class.append(best_prediction)

In [98]:
predicted_class = []
for i in range(predicted_pro.shape[0]):
    best_prediction = np.argsort(predicted_pro[i])[::-1][0] #plus one to get the class label but not the index
    predicted_class.append(best_prediction)

In [204]:
onehot_test_label = enc.transform(test_label.reshape(-1,1))

In [205]:
model.evaluate(train_img,onehot_train_label)

2128/2128 [==============================] - ETA:  - 0s 120us/step


[0.028725174070104845,
 0.9981203007518797,
 0.9981203007518797,
 1.0,
 1.0,
 1.0,
 1.0]

In [203]:
model.evaluate(test_img,onehot_test_label)

766/766 [==============================] - 1s 756us/step


[0.2967305630056416,
 0.9138381191706845,
 0.9138381191706845,
 0.9621409921671018,
 0.9699738903394256,
 0.9765013054830287,
 0.9856396866840731]

In [116]:
train_acc = accuracy_score(np.array(train_label), np.array(train_predicted_class))
train_acc

0.9981203007518797

In [99]:
acc = accuracy_score(np.array(test_label), np.array(predicted_class))
acc

0.9138381201044387

In [110]:
rec = recall_score(np.array(test_label), np.array(predicted_class),average='micro')
rec

0.9138381201044387

In [111]:
pre = precision_score(np.array(test_label), np.array(predicted_class),average='micro')
pre

0.9138381201044387

In [100]:
model.save('res_%s.h5' % str(acc))

In [117]:
predicted_class = np.array(predicted_class)
train_predicted_class = np.array(train_predicted_class)

In [118]:
train_acc_ids = []
for i in range(20):
    acc_id = train_predicted_class[(train_label == i)]
    p = np.sum(acc_id == i)/(len(acc_id))
    train_acc_ids.append(p)

In [119]:
train_acc_ids

[1.0,
 1.0,
 1.0,
 0.9895833333333334,
 0.9913793103448276,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9896907216494846,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9908256880733946]

In [112]:
acc_ids = []
for i in range(20):
    acc_id = predicted_class[(test_label == i)]
    p = np.sum(acc_id == i)/(len(acc_id))
    acc_ids.append(p)

In [113]:
acc_ids

[0.8709677419354839,
 1.0,
 0.8387096774193549,
 0.9130434782608695,
 0.8,
 1.0,
 0.9333333333333333,
 0.9772727272727273,
 1.0,
 0.7948717948717948,
 1.0,
 0.9384615384615385,
 0.8571428571428571,
 0.8076923076923077,
 0.9347826086956522,
 0.9444444444444444,
 0.95,
 0.9565217391304348,
 0.9649122807017544,
 0.9272727272727272]

In [145]:
train_author = new_train_doc[:,1]
uni_train_author = np.unique(train_author)
test_author = new_test_doc[:,1]
uni_test_author = np.unique(test_author)
all_author = np.concatenate((train_author,test_author))
uni_all_author = np.unique(all_author)
print (uni_train_author.shape)
print (uni_test_author.shape)
print (uni_all_author.shape)

(294,)
(266,)
(515,)


In [195]:
pu = []
test_md = new_test_doc[:,3]
for u in uni_test_author:
    iu = np.sum(test_author == u)
    author_test_doc = new_test_doc[(test_author == u)]
    author_test_md = author_test_doc[:,3]
    ranks = []
    for m in author_test_md:
        idx = np.where(test_md == m)[0][0]
        img = test_img[idx].reshape(1,1,1,1,2048)
        true = test_label[idx]
        pro = model.predict(img)
        one_rank = np.where(np.argsort(pro[0])[::-1] == true)[0][0] + 1
        d_rank = 1/one_rank
        ranks.append(d_rank)
    ranks = np.array(ranks)
    mean_r = np.mean(ranks)
    #print (mean_r)
    pu.append(mean_r)
np.mean(np.array(pu))

0.9435835270587531

In [196]:
pu = []
test_md = new_test_doc[:,3]
test_id = new_test_doc[:,0]
test_a = new_test_doc[:,1]
for u in uni_test_author:
    iu = np.sum(test_author == u)
    author_test_doc = new_test_doc[(test_author == u)]
    author_test_md = author_test_doc[:,3]
    ranks = []
    for m in author_test_md:
        idx = np.where(test_md == m)[0][0]
        a  = test_a[idx]
        t_id = test_id[idx]
        img_set = new_test_doc[(new_test_doc[:,0] == t_id)&(new_test_doc[:,1] == a)]
        img_set_md = img_set[:,3]
        nn = []
        for mm in img_set_md:
            idxm = np.where(test_md == mm)[0][0]
            truem = test_label[idxm]
            imgm = test_img[idxm].reshape(1,1,1,1,2048)
            prom = model.predict(imgm)
            one_rankm = np.where(np.argsort(prom[0])[::-1] == truem)[0][0] + 1
            #print (one_rankm)
            d_rankm = 1/one_rankm
            nn.append(d_rankm)
        nn = np.array(nn)
        #print (nn.shape)
        r = np.mean(nn)
        #print (r)
        ranks.append(r)
        #print (ranks)
    ranks = np.array(ranks)
    mean_r = np.mean(ranks)
    #print (mean_r)
    pu.append(mean_r)
np.mean(np.array(pu))

0.9435835270587531